In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [901 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
!pip install -U pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 35.3 MB 135 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-03 22:33:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.31MB/s    in 0.2s    

2022-08-03 22:33:02 (5.31 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url ="https://group13-bucket.s3.us-east-2.amazonaws.com/ONTIME_REPORTING_01_TC.csv"
spark.sparkContext.addFile(url)
ontimereporting_df = spark.read.csv(SparkFiles.get("ONTIME_REPORTING_01_TC.csv"), sep=",", header=True, inferSchema=True)

In [6]:
# Show DataFrame
ontimereporting_df.show()

+-----+------------+---------+-----------+-----------------+--------+-----------------+-----------------+------+----------------+---------------+----+--------------+------------+--------+-------------+---------+------------+------------+--------+-------------+------------+---------+-----------------+----------------+-------------------+--------+--------------+-------------+-------------+---------+--------------+-------------------+
|MONTH|DAY_OF_MONTH|     DATE|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|ORIGIN|ORIGIN_CITY_NAME|DEST_AIRPORT_ID|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|DEP_DELAY_NEW|DEP_DEL15|DEP_TIME_BLK|CRS_ARR_TIME|ARR_TIME|ARR_DELAY_NEW|ARR_TIME_BLK|CANCELLED|CANCELLATION_CODE|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|DISTANCE|DISTANCE_GROUP|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+-----+------------+---------+-----------+-----------------+--------+-----------------+-----------------+------+----------------

In [11]:
final_ontimereporting_df = ontimereporting_df.select(["month", "day_of_month", "date", "day_of_week", "origin_airport_id", "dest_airport_id", "dest_city_name",
"dep_time","dep_delay_new", "arr_time", "arr_delay_new", "dep_del15", "cancelled", "distance", "distance_group"])
final_ontimereporting_df.show()


+-----+------------+---------+-----------+-----------------+---------------+--------------+--------+-------------+--------+-------------+---------+---------+--------+--------------+
|month|day_of_month|     date|day_of_week|origin_airport_id|dest_airport_id|dest_city_name|dep_time|dep_delay_new|arr_time|arr_delay_new|dep_del15|cancelled|distance|distance_group|
+-----+------------+---------+-----------+-----------------+---------------+--------------+--------+-------------+--------+-------------+---------+---------+--------+--------------+
|    1|           6| 1-6-2019|          7|            10397|          11150|   Columbus GA|    1643|            0|    1720|            0|        0|        0|      83|             1|
|    1|           7| 1-7-2019|          1|            10397|          11150|   Columbus GA|    1640|            0|    1712|            0|        0|        0|      83|             1|
|    1|           8| 1-8-2019|          2|            10397|          11150|   Columbus GA

In [7]:
url ="https://group13-bucket.s3.us-east-2.amazonaws.com/airport_weather_2019.csv"
spark.sparkContext.addFile(url)
weather_df = spark.read.csv(SparkFiles.get("airport_weather_2019.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
weather_df.show()

+-----------+--------------------+---------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|    STATION|                NAME|     DATE| AWND|PGTM|PRCP|SNOW|SNWD|TAVG|TMAX|TMIN|WDF2|WDF5|WSF2|WSF5|WT01|WT02|WT03|WT04|WT05|WT06|WT07|WT08|WT09|WESD|WT10|PSUN|TSUN|SN32|SX32|TOBS|WT11|
+-----------+--------------------+---------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|USW00013874|ATLANTA HARTSFIEL...| 1-1-2019|  4.7|null|0.14| 0.0| 0.0|  64|  66|  57| 310| 310|15.0|19.0|   1|null|null|null|null|null|null|null|null|null|null|null|null|null|null|null|null|
|USW00013874|ATLANTA HARTSFIEL...| 1-2-2019| 4.92|null|0.57| 0.0| 0.0|  56|  59|  49|  70|  50|12.1|15.0|   1|null|null|null|null|null|null|   1|null|null|null|null|null|null|null|null|null|
|USW00013874|ATLANTA HARTSFIEL...| 1-3-2019| 

In [15]:
final_weather_df = weather_df.select(["date", "awnd", "prcp", "tavg", "wdf2", "wdf5", "wsf2", "wsf5"])
final_weather_df.show()

+---------+-----+----+----+----+----+----+----+
|     date| awnd|prcp|tavg|wdf2|wdf5|wsf2|wsf5|
+---------+-----+----+----+----+----+----+----+
| 1-1-2019|  4.7|0.14|  64| 310| 310|15.0|19.0|
| 1-2-2019| 4.92|0.57|  56|  70|  50|12.1|15.0|
| 1-3-2019| 5.37|0.15|  52| 340| 330|15.0|18.1|
| 1-4-2019|12.08|1.44|  56| 260| 260|28.0|34.0|
| 1-5-2019|13.42| 0.0|  49| 280| 330|23.9|33.1|
| 1-6-2019| 6.49| 0.0|  53| 320| 320|13.0|15.0|
| 1-7-2019| 5.14| 0.0|  55| 160| 160|15.0|19.0|
| 1-8-2019| 8.05| 0.0|  57| 290| 260|16.1|21.9|
| 1-9-2019|17.45| 0.0|  50| 320| 320|28.0|34.0|
|1-10-2019| 13.2| 0.0|  38| 330| 320|23.9|33.1|
|1-11-2019| 6.93| 0.0|  40| 120| 110|12.1|15.0|
|1-12-2019|12.97|0.35|  43|  80|  90|19.9|25.1|
|1-13-2019| 8.05|0.27|  39|  70|  80|17.0|19.9|
|1-14-2019|11.63| 0.0|  45| 330| 330|17.0|23.0|
|1-15-2019|10.74| 0.0|  40| 320| 310|17.0|25.1|
|1-16-2019| 7.61| 0.0|  38| 320| 330|13.0|16.1|
|1-17-2019| 4.03| 0.1|  42| 180| 180| 8.9|13.0|
|1-18-2019| 2.91| 0.0|  50| 220| 220| 8.

In [16]:
# Join the data into one table 
final_df = final_ontimereporting_df.join(final_weather_df, on = "date", how = "inner")
final_df.show()

+--------+-----+------------+-----------+-----------------+---------------+--------------+--------+-------------+--------+-------------+---------+---------+--------+--------------+-----+----+----+----+----+----+----+
|    date|month|day_of_month|day_of_week|origin_airport_id|dest_airport_id|dest_city_name|dep_time|dep_delay_new|arr_time|arr_delay_new|dep_del15|cancelled|distance|distance_group| awnd|prcp|tavg|wdf2|wdf5|wsf2|wsf5|
+--------+-----+------------+-----------+-----------------+---------------+--------------+--------+-------------+--------+-------------+---------+---------+--------+--------------+-----+----+----+----+----+----+----+
|1-6-2019|    1|           6|          7|            10397|          11150|   Columbus GA|    1643|            0|    1720|            0|        0|        0|      83|             1| 8.72| 0.0|  44| 320| 310|25.9|36.9|
|1-6-2019|    1|           6|          7|            10397|          11150|   Columbus GA|    1643|            0|    1720|          

In [18]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ccozswrrwgoz.us-east-2.rds.amazonaws.com:5432/dataviz"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [19]:
 # Write DataFrame to active_user table in RDS
final_df.write.jdbc(url=jdbc_url, table='final_df', mode=mode, properties=config)

Py4JJavaError: ignored